In [ ]:
import warnings
import gc
warnings.filterwarnings('ignore')

In [2]:
import torch
import evaluate
import pandas as pd
from tqdm import tqdm
from peft import PeftModel, PeftConfig
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tqdm.pandas()
gc.collect()
torch.manual_seed(42)

In [ ]:
full_data_test = pd.read_csv('../dataset/full_test_data_summarization.csv')

In [4]:
checkpoint = './model_checkpoint/checkpoint-1576'

In [ ]:
config = PeftConfig.from_pretrained(checkpoint)
base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    device_map={"":0},
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    quantization_config=BitsAndBytesConfig(    
        load_in_4bit=True,
        load_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16
    )
)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
model = PeftModel.from_pretrained(base_model, checkpoint, device_map={"":0})
model.eval()

In [6]:
def create_prompt(sample):
    template = """<s>[INST] Bạn là một trợ lý AI. Bạn sẽ được giao một nhiệm vụ. Hãy tóm lược ngắn gọn nội dung sau bằng tiếng Việt:
{} [/INST]"""
    prompt = template.format(sample)
    return prompt

In [8]:
references = []
predictions = []
def generate_text():
  for batch_1, batch_2 in tqdm(zip(torch.utils.data.DataLoader(full_data_test['context'], batch_size=112, shuffle=False), torch.utils.data.DataLoader(full_data_test['summarization'], batch_size=112, shuffle=False), strict=True), total=int(round(len(full_data_test)/112, 0))):
    prompts = [create_prompt(context) for context in batch_1]
    inputs = tokenizer(prompts, add_special_tokens=True, padding=True, return_tensors="pt").to(device)
    outputs = model.generate(
      **inputs,
      early_stopping=False,
      max_new_tokens=1024,
      temperature=0.7,
      top_p=0.8,
      top_k=50,
      repetition_penalty=1.2,
      pad_token_id=tokenizer.eos_token_id
    )
    references.extend(batch_2)
    predictions.extend([tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=False).split('[/INST]')[1].strip() for output in outputs])
    torch.cuda.empty_cache()

In [ ]:
references = []
predictions = []
def generate_text():
  for batch_1, batch_2 in tqdm(zip(torch.utils.data.DataLoader(full_data_test['context'], batch_size=112, shuffle=False), torch.utils.data.DataLoader(full_data_test['summarization'], batch_size=112, shuffle=False), strict=True), total=int(round(len(full_data_test)/112, 0))):
    prompts = [create_prompt(context) for context in batch_1]
    inputs = tokenizer(prompts, add_special_tokens=True, padding=True, return_tensors="pt").to(device)
    outputs = model.generate(
      **inputs,
      early_stopping=False,
      max_new_tokens=1024,
      temperature=0.7,
      top_k=10,
      penalty_alpha=0.8,
      repetition_penalty=1.2,
      pad_token_id=tokenizer.eos_token_id
    )
    references.extend(batch_2)
    predictions.extend([tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=False).split('[/INST]')[1].strip() for output in outputs])
    torch.cuda.empty_cache()

In [ ]:
generate_text()

In [ ]:
full_data_test['summarization_predictions'] = predictions

In [ ]:
rouge_metric = evaluate.load("rouge")
rouge_scores = rouge_metric.compute(references=references, predictions=predictions, use_stemmer=True, rouge_types=['rouge1', 'rouge2', 'rougeL'])

In [11]:
rouge_scores

{'rouge1': 0.6073709993163005,
 'rouge2': 0.3535813077835422,
 'rougeL': 0.4224855441941776}

In [12]:
full_data_test.to_csv('test_mistral_lora_v2.csv', index=False)